In [15]:
%reset -f
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
from models import TCNmodel
import numpy as np
from random import randrange
from skimage.io import imread_collection
from IPython.core.display import HTML
from torch import nn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

train_sample,frame = torch.load('dataset_train.pt')
#train_sample,frame = torch.load('dataset_test.pt')

torch.cuda.empty_cache()


In [16]:
# net = TCNmodel()
# net=net.to('cuda')
# test = train_sample.reshape(-1,3,360,640)
# inputs = test[0:41,:,:,:] 
# inputs.shape
# inputs = inputs.to('cuda')
# with torch.no_grad():
#     outputs = net(inputs)
    
# torch.cuda.empty_cache()
    




In [17]:
def output_dataset(data,frame, net,device):
    net = net.to(device)
    data = data.reshape(-1,3,360,640)
    i = 0 
    final_output = []
    F,C,W,H = data.shape
    with torch.no_grad():
        for j in range(frame.shape[0]):
            f = frame[j]+i
            inputs = data[i:f,:,:,:]
            data_loader = torch.utils.data.DataLoader(inputs, batch_size = 12, shuffle = False)
            i = f 
            outputs =torch.tensor([]) 
            for j, subdata in enumerate(data_loader):
                subdata= subdata.to(device)
                suboutputs = net(subdata)
                outputs = torch.cat((outputs,suboutputs.cpu()),0)
            final_output.append(outputs)
    torch.cuda.empty_cache()
    return final_output

#looks for nearest neighbour
def cosine(outputs):
    list_frames_outputs = []
    for i in range(len(outputs)):
        output = outputs[i].cpu().detach().numpy() 
        #cosine = cosine_similarity(output)
        cosine = euclidean_distances(output)
        prev_frame = 0
        list = []
        list.append(prev_frame)
        for i in range(len(output-1)):
            cosine[i][i] = 0
            cosine[:,prev_frame] = 0
            test = np.nanargmax(cosine[i])
            prev_frame = test
            list.append(test)
        list_frames_outputs.append(list)
    return list_frames_outputs

# searches for frames between -1 and 1 frame away > so for frame 3 the next frame can be 4 or 2 
def test_accuracy(list,tolerance):
    accuracy = []
    for j in range(len(list)):
        acc = 0
        for i in range(len(list[j])-1):
            if list[j][i+1] in range(list[j][i]-tolerance-1, list[j][i]+tolerance+1):
                acc += 1 
        accuracy.append((acc / (len(list[j])-1))*100)
    return accuracy

def test(outputs):
    list_frames_outputs = cosine(outputs)
    test_accuracy_list = test_accuracy(list_frames_outputs, 1)
    av_accuracy = np.mean(test_accuracy_list)
    return list_frames_outputs , test_accuracy_list, av_accuracy

import cv2
import IPython.display
import numpy as np
import PIL.Image
from matplotlib import animation
from matplotlib import pyplot as plt


def create_animation(images, interval_ms=100, **fig_kwargs):

    # use larger plot by default
    if "figsize" not in fig_kwargs:
        fig_kwargs["figsize"] = (13, 9)

    fig, ax = plt.subplots(**fig_kwargs)
    fig.tight_layout()
    ax.axis("off")
    im = ax.imshow(images[0][:, :, ::-1])
    plt.close()  # this is required to not display the generated image

    def init():
        im.set_data(images[0][:, :, ::-1])  # ::-1: BGR --> RGB

    def animate(i):
        image = images[i]
        if image is not None:
            im.set_data(image[:, :, ::-1])
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(images), interval=interval_ms)

    # disable warning for video creation (anim.to_*())
    import logging

    logging.getLogger("matplotlib.animation").disabled = True

    # from IPython.core.display import HTML
    # use HTML(anim.to_html5_video()) to show within jupyter notebook as video
    # or HTML(anim.to_jshtml()) to show within jupyter notebook as interactive widget
    return anim


def draw_bbox_to_image(image, bbox, color=(0, 255, 0), thickness=5):
    corner_coords = bbox.get_bbox_corners_vis()
    image = cv2.rectangle(image, corner_coords[:2], corner_coords[-2:], color, thickness)
    return image


def showimage(a):
    """Show an image below the current jupyter notebook cell.
    Expects gray or bgr input (opencv2 default)"""
    # bgr -> rgb
    if len(a.shape) > 2 and a.shape[2] == 3:
        a = a[..., ::-1]  # bgr -> rgb
    image = PIL.Image.fromarray(a)
    IPython.display.display(image)  # display in cell output


# https://colorbrewer2.org/#type=qualitative&scheme=Paired&n=12
colors_qualitative = np.array(
    [
        [166, 206, 227],
        [31, 120, 180],
        [178, 223, 138],
        [51, 160, 44],
        [251, 154, 153],
        [227, 26, 28],
        [253, 191, 111],
        [255, 127, 0],
        [202, 178, 214],
        [106, 61, 154],
        [255, 255, 153],
        [177, 89, 40],
    ]
)

# rgb representation for k3d
colors_qualitative_k3d = np.dot(colors_qualitative, np.asarray([2 ** 16, 2 ** 8, 2 ** 0])).tolist()


In [18]:
# for i in range(5):
#     PATH = 'C:/Users/maxwa/Documents/TCN_self_supervised_learning/SaveModel_' +str(9*i) + '.pth'
#     print(PATH)

In [20]:
%%time
# for i in range(5):
#     PATH = './checkpoint/SaveModel_' +str(159) + '.pth'
#     print(PATH)
PATH = './checkpoints/SaveModel_590.pth'
model = TCNmodel()
model.load_state_dict(torch.load(PATH)['net'])
model.eval()
device = 'cuda'
outputs = output_dataset(train_sample,frame,model,device)
list_frames_outputs, accuracy,av_accuracy = test(outputs)
print(av_accuracy)

Using cache found in C:\Users\Think/.cache\torch\hub\pytorch_vision_v0.10.0


51.20737721764115
Wall time: 6.02 s


In [24]:
print((list_frames_outputs[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
print(av_accuracy)
col_dir_test = 'frames/train/1/*.jpg'
#creating a collection with the available images
img_test = imread_collection(col_dir_test)
print(len(img_test))
print(len(list_frames_outputs[0]))
print(list_frames_outputs[0])
images = []
for i in range(len(list_frames_outputs[0])):
    index = list_frames_outputs[0][i]
    images.append(img_test[index])
    
anim = create_animation(images,interval_ms=100)
video = anim.to_html5_video()
HTML(video)

49.193009476072206
0
42
[0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


IndexError: There are only 0 images in the collection

In [ ]:
col_dir_test = 'frames/test/*.jpg'
#creating a collection with the available images
img_test = imread_collection(col_dir_test)
img_test_1 = img_test[:46]
print(len(img_test_1))
print(img_test_1[0].shape)
test = np.zeros([46,1080,1920,3])
for i in range(46):
    test[i,:,:,:] = img_test_1[i]
    
print(test.shape)
test = np.reshape(test,(46,1080*1920*3))
print(test.shape)
#list_frames_outputs, accuracy,av_accuracy  = test(test)